In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC3
from selenium.webdriver.common.action_chains import ActionChains
from concurrent.futures import ThreadPoolExecutor
from requests.adapters import HTTPAdapter
import requests
import time
from requests_html import HTMLSession
import os

op = webdriver.ChromeOptions()
# op.add_argument('--headless=new')
prefs = {
    'profile.default_content_settings.popups': 0,
    'download.default_directory' : r"/home/administrator/cbs_bag_hold/data",
    'directory_upgrade': True
}
op.add_experimental_option('prefs' , prefs)
driver = webdriver.Chrome(options=op)


session = HTMLSession()


def login_flo():
    driver.get("http://10.24.1.71/mh-ops")
    time.sleep(5)
    username = driver.find_element(By.XPATH , "/html/body/div[2]/div[2]/div/div/form/div/div[4]/input[1]")
    username.send_keys("ca.2670054")

    password = driver.find_element(By.XPATH , "/html/body/div[2]/div[2]/div/div/form/div/div[4]/input[2]")
    password.send_keys("Chauhan@8091")
    time.sleep(2)
    try:
        cross = driver.find_element(By.XPATH , "/html/body/div[4]/div/button")
        cross.click()
    except:
        print("Cross Button Failed")
    time.sleep(1)
    submit = driver.find_element(By.XPATH , "/html/body/div[2]/div[2]/div/div/form/div/div[4]/div[4]/button/span")
    submit.click()
    time.sleep(10)

login_flo()
session_cookie = driver.get_cookies()
csrf_token = driver.execute_script("return document.querySelector('meta[name=csrf-token]').getAttribute('content');")

if csrf_token:
    session.headers.update({"csrf-token": csrf_token})
    print(csrf_token)
else:
    print("CSRF Token not found in page")
print(session_cookie)
selenium_user_agent = driver.execute_script("return navigator.userAgent;")
print(selenium_user_agent)
session.headers.update({"user-agent": selenium_user_agent})
for cookie in driver.get_cookies():
    session.cookies.set(cookie['name'], cookie['value'], domain=cookie['domain']) # Can't is set this here
   

with open("a.txt" , 'r') as f:
    shipments = [line.strip() for line in f if line.strip()]
payload = {'shipment_ids': shipments}


url = "http://10.24.1.71/mh-ops-routes-api/mh-ops-proxy/v1/track-and-trace-proxy/shipment/bulk-track/generate-request-id"
response = session.post(url , json=payload)

if response.status_code == 200:
    response_json = response.json()
    request_tracking_id = response_json["data"]["request_tracking_id"]
    print(request_tracking_id)
    csv_url = f"http://10.24.1.71/mh-ops-routes-api/mh-ops-proxy/v1/track-and-trace-proxy/shipment/bulk-download/{request_tracking_id}"
    print(csv_url)
    if csv_url:
        csv_response = session.get(csv_url)
        if csv_response.status_code == 200:
            download_directory = "/home/flipkart/auto_print/input"
            with open(os.path.join(download_directory, "downloaded_file.csv"), "wb") as csv_file:
                csv_file.write(csv_response.content)
            print("CSV file downloaded successfully")
        else:
            print(f"Failed to download CSV file. Status code: {csv_response.status_code}")
    else:
        print("CSV URL not found in the response")
else:
    # Print an error message if the request was not successful
    print(f"Request failed with status code {response.status_code}")


Cross Button Failed
DWCC82A3-HFVa7toHG7EOdB6ytIpQ9hsEf0g
[{'domain': '10.24.1.71', 'expiry': 1742397112, 'httpOnly': False, 'name': '_ga_SV3K9FT76Q', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 'GS1.1.1707837111.1.1.1707837112.0.0.0'}, {'domain': '10.24.1.71', 'expiry': 1707923510, 'httpOnly': True, 'name': 'session', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 'eyJtaC1vcHMiOnsiX19jdXJyZW50VVJMIjoiL21oLW9wcyIsInN0YXRlIjoiNzllNDdhYmMtYWJjMi00YzEwLTg4MzQtOWJkMDBhODI1ZmZkIiwibm9uY2UiOiJhMGNjOTZjNy0xMzdiLTRhMWMtYTFlZi0yNTY5YmU2ZjlhNzkiLCJ1c2VyIjp7InN1YiI6ImNhLjI2NzAwNTQiLCJuYW1lIjoiY2EuMjY3MDA1NCIsInVzZXJJZCI6ImNhLjI2NzAwNTQifSwiZXhwIjoxNzA3OTE5OTEwMzU5fSwiY3NyZlNlY3JldCI6ImtxdnljSXNDaTcyNk9DdEMxR1djc2M4eiJ9'}, {'domain': '10.24.1.71', 'expiry': 1742397111, 'httpOnly': False, 'name': '_ga', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 'GA1.1.585273674.1707837111'}, {'domain': '10.24.1.71', 'expiry': 1707923510, 'httpOnly': True, 'name': 'session.sig